# Seq2Seq
### Neural Machine Translation using word level language model and embeddings in Keras

- https://medium.com/@dev.elect.iitd/neural-machine-translation-using-word-level-seq2seq-model-47538cba8cd7
- https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py
- https://machinelearningmastery.com/define-encoder-decoder-sequence-sequence-model-neural-machine-translation-keras/
- https://machinelearningmastery.com/develop-encoder-decoder-model-sequence-sequence-prediction-keras/

---

In [1]:
import pandas as pd
import numpy as np

import string
from itertools import chain
import pickle
import re

from string import digits

import matplotlib.pyplot as plt

# Load Data

In [2]:
# load
with open('df_story_desc_final.pickle', 'rb') as f:
    df_story_and_desc = pickle.load(f)

print ("before: ", df_story_and_desc.shape)
df_story_and_desc = df_story_and_desc[:10000]

df_story_and_desc_id = df_story_and_desc['Story_Photo_id']
df_story_and_desc_text = df_story_and_desc[['Desc', 'Story']]

print ("after: ", df_story_and_desc_text.shape)
df_story_and_desc_text.head(15)

before:  (145116, 4)
after:  (10000, 2)


,Desc,Story
0,big old tree being photographed on a sunny day,"and its magnificent trunk , larger than life i..."
1,a old curvy tree in the sun light .,"and its magnificent trunk , larger than life i..."
2,a person is taking a picture of a large tree a...,"and its magnificent trunk , larger than life i..."
3,large tree with many outstretching branches an...,we found this tree when we were walking in a n...
4,a green sign is describing a historic tree and...,it turns out it is a popular attraction here .
5,a large tree with roots that look like crocodi...,"the tree is very unusual , with its roots expo..."
6,big old tree being photographed on a sunny day,"the trunk was really wide , as much as 12 feet !"
7,huge brown tree roots rose above the ground .,you can see how big these roots are - pretty a...
8,a large tree with many branches coming out,we found this tree when we were walking in a n...
9,a plaque describes an historical tree and advi...,it turns out it is a popular attraction here .


==> 15개 단위로 같은 사진 set에 대한 내용

In [3]:
df_story_and_desc_text[df_story_and_desc_text.Desc=='big old tree being photographed on a sunny day']

,Desc,Story
0,big old tree being photographed on a sunny day,"and its magnificent trunk , larger than life i..."
6,big old tree being photographed on a sunny day,"the trunk was really wide , as much as 12 feet !"
18,big old tree being photographed on a sunny day,some more different parts of the tree .
24,big old tree being photographed on a sunny day,the trunk was incredibly thick and rigid .
39,big old tree being photographed on a sunny day,i was dwarfed by the tree 's size .


In [4]:
for i,row in df_story_and_desc_text[df_story_and_desc_text.Desc=='big old tree being photographed on a sunny day'].iterrows():
    print("pair",i,":",row['Desc']+" ==> "+row['Story'])

pair 0 : big old tree being photographed on a sunny day ==> and its magnificent trunk , larger than life itself .
pair 6 : big old tree being photographed on a sunny day ==> the trunk was really wide , as much as 12 feet !
pair 18 : big old tree being photographed on a sunny day ==> some more different parts of the tree .
pair 24 : big old tree being photographed on a sunny day ==> the trunk was incredibly thick and rigid .
pair 39 : big old tree being photographed on a sunny day ==> i was dwarfed by the tree 's size .


==> 하나의 Description Text에 5개의 Story가 대응됨

## cleaning

In [5]:
def re_sub(item):
    re_sentence = []
    for sentence in item:
        sentence = re.sub('[^a-z0-9A-Z]+', ' ', sentence)
        re_sentence.append(sentence)
    return re_sentence

In [6]:
clean_data = df_story_and_desc_text.apply(lambda x: re_sub(x))
clean_data.columns = ['in_desc','out_story']
clean_data['out_story'] = clean_data['out_story'].apply(lambda x : '<sos> '+ x + ' <eos>')
clean_data

,in_desc,out_story
0,big old tree being photographed on a sunny day,<sos> and its magnificent trunk larger than li...
1,a old curvy tree in the sun light,<sos> and its magnificent trunk larger than li...
2,a person is taking a picture of a large tree a...,<sos> and its magnificent trunk larger than li...
3,large tree with many outstretching branches an...,<sos> we found this tree when we were walking ...
4,a green sign is describing a historic tree and...,<sos> it turns out it is a popular attraction ...
...,...,...
9995,a man in glasses that is standing next to people,<sos> we all visited a winery and took a tour ...
9996,people in a room wit ha bottle of wine,<sos> they took us inside and let us see where...
9997,a man is sniffing a glass of clear champagne,<sos> it was amazing to taste wine while stand...
9998,people are seated at a long outside table unde...,<sos> we all sat down together for a meal <eos>


# Vectorize the data

--- 

0907 DONE

---

In [7]:
desc_words = set()
for desc_paragraph in clean_data.in_desc:
    for word in desc_paragraph.split():
        if word not in desc_words:
            desc_words.add(word)
            
story_words = set()
for story_parapgraph in clean_data.out_story:
    for word in story_parapgraph.split():
        if word not in story_words:
            story_words.add(word)
    
len(desc_words), len(story_words)

(2865, 3486)

In [8]:
input_words = sorted(list(desc_words))
target_words = sorted(list(story_words))

num_encoder_tokens = len(desc_words)
num_decoder_tokens = len(story_words)

max_encoder_seq_length = max([len(txt.split(" ")) for txt in clean_data['in_desc']])
max_decoder_seq_length = max([len(txt.split(" ")) for txt in clean_data['out_story']])

print('Number of samples:', len(input_words))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 2865
Number of unique input tokens: 2865
Number of unique output tokens: 3486
Max sequence length for inputs: 45
Max sequence length for outputs: 47


In [9]:
len(clean_data.out_story), max_decoder_seq_length, num_decoder_tokens

(10000, 47, 3486)

In [10]:
encoder_input_data = np.zeros(
    (len(clean_data.in_desc), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(clean_data.out_story), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(clean_data.out_story), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

메모리 뻑남..

MemoryError: Unable to allocate 59.8 GiB for an array with shape (30000, 82, 6524) and data type float32


In [11]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 45)
(10000, 47)
(10000, 47, 3486)


**why decoder_target_data.shape is 3d**

- 모든 단어에 대하여 이전 단어로부터 다음 단어를 예측하는 소프트맥스 층을 가지기 때문에 

--- 

0908 DONE

- 왜 1로 초기화 하는가

    - ==> 초기화를 1로 하는 것이 아니라 t=0을 건너뛰는 것임

---

In [12]:
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])

In [29]:
for i, (input_text, target_text) in enumerate(zip(clean_data['in_desc'], clean_data['out_story'])):

    # encoder
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]

    # decoder
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]  
        if t > 0: 
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [30]:
encoder_input_data.shape

(10000, 45)

In [31]:
print (decoder_input_data.shape)
decoder_input_data

(10000, 47)


array([[  27.,  125., 1676., ...,    0.,    0.,    0.],
       [  27.,  125., 1676., ...,    0.,    0.,    0.],
       [  27.,  125., 1676., ...,    0.,    0.,    0.],
       ...,
       [  27., 1674., 3347., ...,    0.,    0.,    0.],
       [  27., 3368.,   96., ...,    0.,    0.,    0.],
       [  27., 3368., 1096., ...,    0.,    0.,    0.]], dtype=float32)

In [32]:
print(decoder_target_data.shape)
decoder_target_data

(10000, 47, 3486)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

--- 

0909 DONE

---

# Build keras encoder-decoder model

http://incredible.ai/nlp/2020/02/20/Sequence-To-Sequence-with-Attention/
https://docs.chainer.org/en/stable/examples/seq2seq.html

https://tykimos.github.io/2018/09/14/ten-minute_introduction_to_sequence-to-sequence_learning_in_Keras/

In [16]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional

In [17]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
embedding_size = 150 # Embedding Dimension

In [36]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
print('encoder_inputs: ', encoder_inputs.shape)

# English words embedding
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
print('en_x: ', en_x.shape)

encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
print('encoder_outputs: ', encoder_outputs.shape)
print('state_h: ', state_h.shape)
print('state_c: ', state_c.shape)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

encoder_inputs:  (None, None)
en_x:  (None, None, 150)
encoder_outputs:  (None, 256)
state_h:  (None, 256)
state_c:  (None, 256)


In [39]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, ))
print('decoder_inputs: ', decoder_inputs.shape)

# french word embeddings
de_x=  Embedding(num_decoder_tokens, embedding_size)(decoder_inputs)

# We set up our decoder to return full output sequences, and to return internal states as well. 
# We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(de_x,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
print('decoder_outputs: ', decoder_outputs.shape)

decoder_inputs:  (None, None)
decoder_outputs:  (None, None, 3486)


--- 

0911 DONE

---

In [40]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 150)    429750      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 150)    522900      input_8[0][0]                    
_______________________________________________________________________________________

In [41]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 45)
(10000, 47)
(10000, 47, 3486)


In [42]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 69s 555ms/step - loss: 1.3465 - accuracy: 0.0297 - val_loss: 1.5454 - val_accuracy: 0.0324
Epoch 2/100
125/125 [==============================] - 67s 534ms/step - loss: 1.2301 - accuracy: 0.0387 - val_loss: 1.4888 - val_accuracy: 0.0421
Epoch 3/100
125/125 [==============================] - 61s 489ms/step - loss: 1.1443 - accuracy: 0.0469 - val_loss: 1.4519 - val_accuracy: 0.0454
Epoch 4/100
125/125 [==============================] - 61s 490ms/step - loss: 1.0810 - accuracy: 0.0516 - val_loss: 1.4308 - val_accuracy: 0.0494
Epoch 5/100
125/125 [==============================] - 61s 486ms/step - loss: 1.0259 - accuracy: 0.0549 - val_loss: 1.4160 - val_accuracy: 0.0497
Epoch 6/100
125/125 [==============================] - 60s 479ms/step - loss: 0.9742 - accuracy: 0.0580 - val_loss: 1.4193 - val_accuracy: 0.0509
Epoch 7/100
125/125 [==============================] - 57s 458ms/step - loss: 0.9250 - accuracy: 0.0609 - val_loss: 1.4203 -

결과가.. 이상..

In [ ]:
# Save model
model.save('s2s.h5')

# Sentence Generation

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class AttentionL(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(AttentionL, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

    def get_config(self):
        config={'step_dim':self.step_dim}
        base_config = super(AttentionL, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# Model test

In [ ]:
encoder_inputs = Input(shape=(None,))

In [ ]:
# 11-02  decoder 1 layer
encoder_inputs = Input(shape=(None,))

display('encoder_inputs: ', encoder_inputs.shape)

en_x=  Embedding(num_encoder_tokens, 150)(encoder_inputs)

encoder = Bidirectional(LSTM(100, return_state=True,
                             dropout = 0.5, recurrent_dropout = 0.5))

encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(en_x)

# state_h = Concatenate()([forward_h, backward_h])
# state_c = Concatenate()([forward_c, backward_c])

# # We discard `encoder_outputs` and only keep the states.
# encoder_states = [state_h, state_c]

# ### decoder

# decoder_inputs = Input(shape=(None,))

# dex=  Embedding(num_decoder_tokens, EMBEDDING_DIM)

# final_dex= dex(decoder_inputs)

# decoder_lstm = LSTM(100 * 2, return_sequences=True, return_state=True,
#                     dropout = 0.5, recurrent_dropout = 0.5)

# print (decoder_lstm(final_dex, initial_state = encoder_states))
# decoder_outputs, _, _ = decoder_lstm(final_dex, initial_state = encoder_states)

# decoder_dense = Dense(num_decoder_tokens, activation='softmax')

# decoder_outputs = decoder_dense(decoder_outputs)

# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc'])

# model.summary()

In [ ]:
import keras.backend.tensorflow_backend as K
with K.tf.device('/gpu:0'):
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size = 128,
              epochs = 50)

In [ ]:
# from keras.models import load_model
# model.save("1124_bilstm_emb150_model22.h5")

#### Create sampling model

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

In [ ]:
decoder_state_input_h = Input(shape=(units * 2,))  # encoder를 bilstm으로 학습했기 때문에 shape이 50이 아니고 100이다.
decoder_state_input_c = Input(shape=(units * 2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)
print (final_dex2)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [ ]:
# # save
# with open('bilstm150_encoder_model_1124.json', 'w', encoding='utf8') as f:
#     f.write(encoder_model.to_json())
# encoder_model.save_weights('bilstm150_encoder_model_1124.h5')

# with open('bilstm150_decoder_model_1124.json', 'w', encoding='utf8') as f:
#     f.write(decoder_model.to_json())
# decoder_model.save_weights('bilstm150_decoder_model_1124.h5')

In [ ]:
# model load
from keras.models import model_from_json
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = model_from_json(f.read())
    model.load_weights(model_weights_filename)
    return model

encoder = load_model('test_bilstm150_encoder_model.json', 'test_bilstm150_encoder_model.h5')
decoder = load_model('test_bilstm150_decoder_model.json', 'test_bilstm150_decoder_model.h5')

In [ ]:
reverse_input_char_index

In [ ]:
reverse_target_char_index

In [ ]:
target_token_index['START_']

#### Function to generate sequences

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    #print ("target_seq: ", target_seq)
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        #print ("output_tokens: ", output_tokens)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        #print ("sampled_token_index: ", sampled_token_index)
        #print ("")
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 150): # 52
            # print ("Stop_condition = TRUE")
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# test data

In [ ]:
df_input5000_0 = clean_df_all.description_text[0:5000]
df_input5000_1 = clean_df_all.description_text[5000:10000]
df_input5000_2 = clean_df_all.description_text[10000:15000]
df_input5000_3 = clean_df_all.description_text[15000:20000]
df_input5000_4 = clean_df_all.description_text[20000:25000]
df_input5000_5 = clean_df_all.description_text[25000:30000]
df_input5000_5

In [ ]:
df_story_and_desc_id

In [ ]:
def return_input_length(start_num, data):
    df_input_length = []
    for i in range(start_num, start_num + len(data)):
        df_input_length.append(i)
    return df_input_length

df_input5000_0_length = return_input_length(0, df_input5000_0)
df_input5000_1_length = return_input_length(5000, df_input5000_1)
df_input5000_2_length = return_input_length(10000, df_input5000_2)
df_input5000_3_length = return_input_length(15000, df_input5000_3)
df_input5000_4_length = return_input_length(20000, df_input5000_4)
df_input5000_5_length = return_input_length(25000, df_input5000_5)

df_input5000_5_length

In [ ]:
import re
from tqdm import tqdm
# clean_df_all.description_text, clean_df_all.story_text
desc_paragraph = []
story_paragraph = []
#df_input_length_list= [df_input5000_0_length, df_input5000_1_length, df_input5000_2_length, df_input5000_3_length, df_input5000_4_length, df_input5000_5_length]
df_input_length_list = [df_input5000_0_length, df_input5000_1_length, df_input5000_2_length]

for item in tqdm(df_input_length_list): 
    for seq_index in item:
        input_seq = encoder_input_data[seq_index: seq_index + 1]
        decoded_sentence = decode_sequence(input_seq)
        #print('-')
        #print('Input sentence:', clean_df_all.description_text[seq_index: seq_index + 1])
        #print (type(clean_df_all.description_text[seq_index: seq_index + 1]))
        desc_paragraph.append(list(df_test_input3.description_text[seq_index: seq_index + 1]))
        #print('decoded sentence: ', decoded_sentence)
        re_decoded_sentence = re.sub('_END', '', decoded_sentence).strip()
        #print('Re decoded sentence:', re_decoded_sentence)
        story_paragraph.append(re_decoded_sentence)